In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from sklearn.metrics import accuracy_score

> Importing libraries

In [ ]:
import ipywidgets as widgets 
import io 
from PIL import Image
import tqdm
from sklearn.model_selection import train_test_split
import cv2
from sklearn.utils import shuffle 
import tensorflow as tf


folder paths

In [ ]:
import cv2
import os
import numpy as np

X_train = []
Y_train = []
image_size = 150
labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

for label in labels:
    folderPath = os.path.join('/kaggle/input/brain-tumor-classification-mri/Training', label)
    for file in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath, file))
        if img is not None:
            img = cv2.resize(img, (image_size, image_size))
            X_train.append(img)
            Y_train.append(label)
        else:
            print(f"Error reading image: {os.path.join(folderPath, file)}")

for label in labels:
    folderPath = os.path.join('/kaggle/input/brain-tumor-classification-mri/Testing', label)
    for file in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath, file))
        if img is not None:
            img = cv2.resize(img, (image_size, image_size))
            X_train.append(img)
            Y_train.append(label)
        else:
            print(f"Error reading image: {os.path.join(folderPath, file)}")

X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
X_train, Y_train = shuffle(X_train, Y_train, random_state=101)
X_train.shape


train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size =0.1, random_state = 101)

In [ ]:
import tensorflow as tf
import numpy as np

labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

# Convert y_train to one-hot encoded format
y_train_new = []
for label in y_train:
    y_train_new.append(labels.index(label))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)

# Convert y_test to one-hot encoded format
y_test_new = []
for label in y_test:
    y_test_new.append(labels.index(label))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)


Convolutional Nueral Network


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape=(150, 150, 3)))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0,3))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(Dropout(0,3))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0,3))
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0,3))
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(Conv2D(256, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0,3))
model.add(Flatten())
model.add(Dense(512, activation ='relu'))
model.add(Dense(512, activation ='relu'))
model.add(Dropout(0,3))
model.add(Dense(4, activation = 'softmax'))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras import metrics

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[
    metrics.CategoricalAccuracy(),
    metrics.Precision(),
    metrics.Recall(),
    metrics.AUC(name='auc')
])

In [ ]:
history = model.fit(X_train, y_train, epochs = 20, validation_split = 0.1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

# Save the model
model.save('braintumor.h5')

# Get the accuracy values from the training history
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']

epochs = range(len(acc))

# Plot the training accuracy
plt.figure(figsize=(14, 7))
plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.legend(loc='upper left')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.show()

# Plot the validation accuracy
plt.figure(figsize=(14, 7))
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.legend(loc='upper left')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Save the model
model.save('braintumor.h5')

# Get the loss values from the training history
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

# Plot the training loss
plt.figure(figsize=(14, 7))
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.legend(loc='upper right')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

# Plot the validation loss
plt.figure(figsize=(14, 7))
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.legend(loc='upper right')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Validation Loss')
plt.show()

Prediction

In [ ]:
img = cv2.imread('/kaggle/input/brain-tumor-classification-mri/Training/meningioma_tumor/m (12).jpg')
img = cv2.resize(img,(150,150))
img_array = np.array(img)
img_array.shape

In [ ]:
img_array = img_array.reshape(1, 150, 150, 3)
img_array.shape

In [ ]:
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

img = image.load_img('/kaggle/input/brain-tumor-classification-mri/Training/meningioma_tumor/m (12).jpg')
plt.imshow(img, interpolation='nearest')
plt.show()

In [ ]:
a = model.predict(img_array)
indices = a.argmax()
indices